In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:



import numpy as np 
import pandas as pd 
import warnings 
warnings.filterwarnings("ignore")
import tensorflow as tf 


print(tf.__version__)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras import Input
from tensorflow.keras.layers import Flatten, Reshape
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from attention import Attention_Block
# from attention import attention_block
from tensorflow.keras.layers import Attention
from step_attention import AttentionAugmentation2D, _normalize_depth_vars
# from keras.models import load_model

# In[2]:


# from google.colab import drive
# drive.mount('/content/drive')



# In[3]:


from loss import discounted_l1, max_displacement_error,v3_displacement_error
import math
import pickle5 as pickle
import matplotlib.pyplot as plt
from numpy import array
# import horovod.tensorflow.keras as hvd

2.2.0


In [2]:

# hvd.init()

# In[4]:

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"


gpus = tf.config.experimental.list_logical_devices('GPU')
print(gpus)


if gpus:
    
    try:
        
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except:
        print('checking gpu error')
print('checking GPUs')


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config=tf.compat.v1.ConfigProto()
# config.gpu_options.visible_device_list= '0,1'
config.gpu_options.allow_growth=True

config.log_device_placement=True
# config.visible_device_list =2
sess=tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(sess)


# In[5]:


from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split

from load_data import load_data
import os

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
checking gpu error
checking GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:05:00.0, compute capability: 6.1



In [3]:

from pathlib import Path
home = str(Path.home())





In [4]:

from pathlib import Path
home = str(Path.home())

In [5]:

# In[9]:


filters=20
kernel_size = (3,3)
strides = (1,1)
depth_k =0.2
depth_v =0.2

channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
print("before normalized k, v",depth_k,depth_k,strides)
depth_k, depth_v = _normalize_depth_vars(depth_k, depth_v, filters)
print("after normalized k, v",depth_k,depth_k,strides)


# In[10]:
time_step=10
tf.compat.v1.disable_eager_execution()

rnn = Sequential()
rnn.add(LSTM(128, return_sequences=True, kernel_regularizer='l1', input_shape=(time_step,2)))
rnn.add(LSTM(256, return_sequences=True, kernel_regularizer='l1',input_shape=(time_step,128)))
rnn.add(Reshape((time_step,16,16,1)))
rnn.summary()



cnn = Sequential()

cnn.add(Conv2D(1024, (4, 4),input_shape=(16, 16, 1)))
cnn.add(Conv2D(512, (2, 2)))
cnn.add(Conv2D(256, (4, 4)))
cnn.add(Conv2D(128, (2, 2)))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(64, (2, 2)))
cnn.add(Conv2D(32, (2, 2)))
cnn.add(Conv2D(16, (2, 2)))

#attention block below
cnn.add(Conv2D(2 * depth_k + depth_k, (1, 1), strides))
cnn.add(AttentionAugmentation2D(depth_k, depth_v, num_heads=4, relative=True))

# cnn.add(Dropout(0.5))
cnn.add(Flatten())

 
cnn.add(Dense(128))
cnn.add(Dense(32))

cnn.summary() 



rnn2=Sequential()
rnn2.add(GRU(256, input_shape=(time_step,32)))


dense = Sequential()
dense.add(Dense(128))
dense.add(Dense(64))
dense.add(Dense(2)) # Model output

main_input = Input(shape=(time_step,2)) 
model = rnn(main_input)
model = TimeDistributed(cnn)(model) 
 # combine timedistributed cnn, attention, and fully connected network with rnn
model = rnn2(model)
model = dense(model) # add dense
final_model = Model(inputs=main_input, outputs=model)
final_model.summary()


before normalized k, v 0.2 0.2 (1, 1)
after normalized k, v 4 4 (1, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           67072     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 256)           394240    
_________________________________________________________________
reshape (Reshape)            (None, 10, 16, 16, 1)     0         
Total params: 461,312
Trainable params: 461,312
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 13, 1024)      17408     
________

In [6]:


n_batch = 32

n_epoch  = 150


# checkpoint_path =home+ "/repo/models/lyft_cyc2_{epoch}.h5"
# checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)


 #(None, n_steps, n_features)

sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.5, nesterov=True)
rmsprop =tf.keras.optimizers.RMSprop(learning_rate=0.001)



ftrl = tf.keras.optimizers.Ftrl(learning_rate=0.01)


mtrc = [tf.keras.metrics.MeanAbsoluteError()]
# [tf.keras.metrics.CosineSimilarity(axis=1)]

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.95)

adagrad = tf.keras.optimizers.Adagrad(learning_rate=lr_schedule)
adadelta = tf.keras.optimizers.Adadelta(learning_rate=lr_schedule)

# # to use customized loss, choose from  discounted_l1 or max_displacement_error function as loss parameter 
# final_model.compile(loss=v3_displacement_error, optimizer=adadelta, metrics=mtrc)
# # final_model.load_weights('/home/lab1/repo/models/lyft_cyclist/lyft_cyclist2_larger_1.h5')

# history=final_model.fit(X_train, 
#                         y_train, epochs=n_epoch, 
#                         batch_size=n_batch, 
#                         verbose=2, 
#                         validation_data=(X_val, y_val),
#                         callbacks=[cp_callback])





In [7]:
# print("Evaluate on test data")
# results = final_model.evaluate(X_test, y_test, batch_size=32)
# print("test loss, test acc:", results)


# _history_path = home+ '/repo/models/cyc2_history.sav'
# pickle.dump(history.history, open(_history_path, 'wb'))

# _weights_path =  home+ '/repo/models/cyc2_weights.h5'
# final_model.save_weights(_weights_path)


In [24]:
final_model.compile(loss=v3_displacement_error, optimizer=adadelta, metrics=mtrc)

# final_model.load_weights('/home/lab1/repo/models/lyft_vehnew_150.h5')
# ground_truth = np.load("array/ground_truth_prediction_vehicle.npy",allow_pickle=True) # load data here
# samples = np.load("array/history_vehicle.npy",allow_pickle=True)

# final_model.load_weights('/home/lab1/repo/models/lyft_pednew_150.h5')
# ground_truth = np.load("array/ground_truth_prediction_pedestrian.npy",allow_pickle=True) # load data here
# samples = np.load("array/history_pedestrian.npy",allow_pickle=True)


# final_model.load_weights('/home/lab1/repo/models/lyft_cycnew005_150.h5')
final_model.load_weights('/home/lab1/repo/models/lyft_cyctransferped_150.h5') # transfer learning from ped
# final_model.load_weights('/home/lab1/repo/models/lyft_cyc_transferveh_150.h5') # transfer learning from veh
ground_truth = np.load("array/ground_truth_prediction_cyclist.npy",allow_pickle=True) # load data here
samples = np.load("array/history_cyclist.npy",allow_pickle=True)


##prediction
# from utilities import get_position_sequence
def get_position_sequence(prev, new):
    arr = np.delete(prev, 0, 0)
    a=np.vstack((arr,new))
    return a 

def get_sequence_prediction(X_position_test, i):
    results =[]
    s=X_position_test[i]
    for count in range(0,10):
        
        result = final_model.predict(np.array([s]))
#         print(result)
        results.append(result[0])
        s = get_position_sequence(s, result)
    results=np.array(results)
#     print(s.shape,results.shape)
    
    return results

# i= np.random.randint(0, len(X_test))
# results = get_sequence_prediction(X_test, i)
# print(results.shape)

# test=np.load('cyclist_test_history.npy')
# v_predict=[]
# print(len(test))
# for i in range(len(test)):
#     print(i)
#     results = get_sequence_prediction(test, i)
#     v_predict.append(results)


In [25]:
# print(np.array(v_predict).shape)
# np.save("c_predictions",v_predict)
# print('done')

In [26]:


samples=samples.astype(float)[:3000]
ground_truth = ground_truth.astype(float)[:3000]
# exception = set(exception.astype(float))
# samples= np.around(samples,decimals=6)
# ground_truth = np.around(ground_truth,decimals=6)
exception = set()

print(samples.shape, ground_truth.shape)

(164, 10, 2) (164, 10, 2)


In [27]:
final_results=[]
final_displacement=[]
avg_displacement=[]
for idx in range(0,len(samples)):
    if idx in exception: 
        print(idx) 
        continue
    if idx%100==0:print(idx)
    p=[]
    re= get_sequence_prediction(samples, idx)
    final_results.append(re)
#     print(re.shape, ground_truth[idx].shape)
#     print(re,ground_truth[idx] )
    distance = v3_displacement_error(re,ground_truth[idx] )
    avg = K.mean(distance)
    
    with tf.compat.v1.Session() as sess:
        p=distance.eval()
        ax =avg.eval()
#     print(p,ax)
    final_displacement.append(p)
    avg_displacement.append(ax)

final_displacement = np.array(final_displacement)
sec=[]

for j in range(0,final_displacement.shape[1]):
    displacement = sum(final_displacement[:,j])/len(final_displacement)
    
    sec.append(displacement)
print("Final displacement error:",sec[-1])
print("Average displacement error:",sum(avg_displacement)/len(avg_displacement))







0
100
Final displacement error: 3.743121159222175
Average displacement error: 1.4308402563846883


In [28]:
np.save("model_cyc_ped",sec)
print(sec)

[0.1280403699476567, 0.3010705987043603, 0.49245525886989894, 0.7115947514986172, 0.9542853649618458, 1.2521581748050092, 1.6496019286759505, 2.1866272567800835, 2.8894477003812877, 3.743121159222175]


In [29]:
final_results=np.array(final_results)
# np.save("vehicle_prediction_scene_16220", final_results)
print(final_results.shape, ground_truth.shape)
print(final_displacement.shape)
np.save("model_cyctransfer_ped_displacement",final_displacement)

(164, 10, 2) (164, 10, 2)
(164, 10)


In [15]:
def compare(y_true,y_pred):
    """
    compute the max displacement in prediction and ground truth
    return a loss keras tensor
    """
  
    print(y_true.shape, y_pred.shape)
    res=[]
    avg=[]
    for index in range(len(y_true)):
        true = y_true[index]
        pred= y_pred[index]
        error=[]
        for p1, p2 in zip(pred, true): # 50 points
            d = math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
            error.append(d)

        avg.append(sum(error)/len(error))
        res.append(error)


    res=np.array(res)
    ade =round(sum(avg)/len(avg),3)
    fde = round(sum(res[:,-1])/len(res),3)

    return ade, fde

y_true=[]
for idx in range(len(ground_truth)):
    if idx in exception:
        continue
    y_true.append(ground_truth[idx])
y_true=np.array(y_true)

ade, fde = compare(final_results, y_true)
print(ade, fde)

(164, 10, 2) (164, 10, 2)
1.431 3.743


In [ ]:
# # i= np.random.randint(0, len(X_test))


# # # In[ ]:
# # # i =2273

# # # i=1574
# # # i=3070

# # # i=387,39
# # # i=667
# # # i=15
# # print("showing trip", i)


# # from utilities import get_position_sequence
# def get_position_sequence(prev, new):
#     arr = np.delete(prev, 0, 0)
#     a=np.vstack((arr,new))
#     return a 

# def get_sequence_prediction(X_position_test, i):
#     results =[]
#     s=X_position_test[i]
#     for count in range(0,12):
        
#         result = final_model.predict(np.array([s]))
# #         print(result)
#         results.append(result[0])
#         s = get_position_sequence(s, result)
#     results=np.array(results)
# #     print(s.shape,results.shape)
    
#     return results

# results = get_sequence_prediction(X_test, i)

# # print(len(results))
# # print("sss",results)

# ax1 =plt.scatter(y_test[i][:,0],y_test[i][:,1],color='b',label='Ground truth')

# ax2 =plt.scatter(X_test[i][:,0],X_test[i][:,1],color='r', label='Training segment')


# ax3 =plt.scatter(results[:,0],results[:,1],color='g', label='Prediction')
# # ax3 =plt.scatter(result[0][0],result[0][1],color='g', label='Prediction')

# plt.legend(handles=[ax1, ax2, ax3])
# plt.show()

# distance = v3_displacement_error(results,y_test[i] )
# avg = K.mean(distance)
# pl =[]
# # tf.print(distance,output_stream=sys.stderr)
# with tf.compat.v1.Session() as sess: pl=distance.eval(), print(distance.eval(),'\naverage',avg.eval()) 
# import matplotlib.pyplot as plt
# # print(pl[0])
# plt.plot(pl[0])
# plt.title('Displacement error')
# plt.show()

In [ ]:
# final_displacement=[]
# avg_displacement=[]

# # for idx in range(0,len(X_test)):
# for idx in range(0,1000):
#     p=[]
#     re= get_sequence_prediction(X_test, idx)
#     distance = v3_displacement_error(re,y_test[idx] )
#     avg = K.mean(distance)
    
#     with tf.compat.v1.Session() as sess:
#         p=distance.eval()
#         ax =avg.eval()
# #     print(p,ax)
#     final_displacement.append(p)
#     avg_displacement.append(ax)

# final_displacement = np.array(final_displacement)
# sec=[]

# for j in range(0,final_displacement.shape[1]):
#     displacement = sum(final_displacement[:,j])/len(final_displacement)
    
#     sec.append(displacement)
# print("Final displacement error:",sec[-1])
# print("Average displacement error:",sum(avg_displacement)/len(avg_displacement))

# print(sec)
# plt.plot(sec)
# plt.title("average displacement error at each timestep")
# plt.show()



In [ ]:
# history2 = pickle.load(open( cwd+ '/saved_model/lab1_txt_max_history_mae_adadelta_600.sav',  "rb") ) 
# history3 = pickle.load(open( cwd+ '/saved_model/lab1_txt_max_history_mae_adadelta.sav',  "rb") )
# print(len(history.history['loss']))
# # plt.plot(range(600,1100),history.history['loss'], c='b')
# # plt.plot( range(300,600), history2['loss'], c='b')
# plt.plot( history3['loss'], c='b')
# plt.title('train loss')

# # plt.plot(range(600,1100),history.history['val_loss'], c='r')
# # plt.plot( range(300,600), history2['val_loss'], c='r')
# plt.plot( history3['val_loss'], c='r')
# plt.title('loss')


# plt.show()